In [1]:
#default_exp ghtop

# ghtop

> I cannot believe this library name is not already taken on pypi.

In [25]:
#export
import time, sys, signal, shutil, os, json, enlighten, emoji, blessed
from ghtop.dashing import *
from collections import defaultdict

from fastcore.utils import *
from fastcore.foundation import *
from fastcore.script import *
from ghapi.all import *

In [3]:
#export
term = Terminal()
logfile = Path("log.txt")

In [4]:
#export
def github_auth_device():
    auth = GhDeviceAuth()
    wb = input(f"""First copy your one-time code: {term.yellow}{auth.user_code}{term.normal}
    Then visit {auth.verification_uri} in your browser, and paste the code when prompted.
    Shall we try to open the link for you? [y/n] """)
    if wb[0].lower()=='y': auth.open_browser()

    print ("Waiting for authorization...", end='')
    token = auth.wait(lambda: print('.', end=''))
    print("Authenticated with GitHub!")
    return token

In [5]:
#export
def exit(msg):
    print(msg, file=sys.stderr)
    sys.exit()

In [6]:
#export
def get_token():
    path = Path.home()/".ghtop_token"
    if path.is_file():
        try: return path.read_text().strip()
        except: exit("Error reading token")

    token = github_auth_device()
    path.write_text(token)
    return token

In [7]:
token = get_token()

In [8]:
#export
def limit_cb(rem,quota):
    w='WARNING '*7
    if rem < 1000: print(f"{w}\nRemaining calls: {rem} out of {quota}\n{w}")

In [9]:
#export
api = GhApi(limit_cb=limit_cb)

In [10]:
#export
def fetch_events(): return api.activity.list_public_events(per_page=100)

def _maybeAttrDict(x): return AttrDict(x) if isinstance(x, dict) else x

In [11]:
#export
def read_json_log():
    try: res = logfile.read_json()
    except FileNotFoundError: return []    
    return L(res).map(dict2obj) if isinstance(res,list) else L([dict2obj(res)])

In [12]:
#export
Events = dict(
    IssuesEvent_closed=('⭐', 'closed', noop),
    IssuesEvent_opened=('📫', 'opened', noop),
    IssueCommentEvent=('💬', 'commented on', term.white),
    PullRequestEvent_opened=('✨', 'opened a pull request', term.yellow),
    PullRequestEvent_closed=('✔', 'closed a pull request', term.green),
)

In [13]:
#export
printed_event_ids = {}

In [14]:
#export
def to_log(e):
    login,repo,pay = e.actor.login,e.repo.name,e.payload
    typ = e.type + (f'_{pay.action}' if e.type in ('PullRequestEvent','IssuesEvent') else '')
    emoji,msg,color = Events.get(typ, [0]*3)
    if emoji:
        xtra = '' if e.type == "PullRequestEvent" else f' issue # {pay.issue.number}'
        d = try_attrs(pay, "pull_request", "issue")
        return color(f'{emoji} {login} {msg}{xtra} on repo {repo[:20]} ("{d.title[:50]}...")')
    elif e.type == "ReleaseEvent": return f'🚀 {login} released {e.payload.release.tag_name} of {repo}'

In [15]:
#export
def print_event(e, commits_counter):
    if e.id in printed_event_ids: return
    printed_event_ids[e.id] = 1
    login = e.actor.login
    if "bot" in login or "b0t" in login: return  # Don't print bot activity (there is a lot!)

    res = to_log(e)
    if res: print(res)
    elif e.type == "PushEvent": [commits_counter.update() for c in e.payload.commits]
    elif e.type == "SecurityAdvisoryEvent": print(term.blink("SECURITY ADVISORY"))

In [16]:
ev = [o for o in fetch_events() if o.type in ('IssuesEvent','ReleaseEvent','PullRequestEvent')]
[(i,o.type) for i,o in enumerate(ev)]
printed_event_ids={}
for e in ev: print_event(e,None)

✨ kenanduman1988 opened a pull request on repo syntaxerrors/Steam ("add php 8.0 support...")
✨ coding-to-music opened a pull request on repo coding-to-music/codi ("first readme generated...")
📫 bigeasy opened issue # 24 on repo bigeasy/r-tree ("Upgrade `reciprocate` to `0.0.2`....")
⭐ DorShahar closed issue # 13270 on repo elementor/elementor ("Responsive Options for 'Layout' Settings...")


In [17]:
#export    
def write_logs(events):
    Path("tmp.log").write_text(json.dumps(events, indent=2))
    shutil.move("tmp.log", logfile)

def tail_events():
    manager = enlighten.get_manager()
    commits = manager.counter(desc='Commits', unit='commits', color='green')
    while True:
        combined = (fetch_events()+read_json_log()).sorted(using_attr(int,"id"))
        write_logs(obj2dict(combined))
        for x in combined: print_event(x, commits)
        time.sleep(0.2)

In [26]:
#export
def watch_users():
    users,users_events = defaultdict(int),defaultdict(lambda: defaultdict(int))
    while True:
        for x in fetch_events():
            login = x.actor.login
            users[login] += 1
            users_events[login][x.type] += 1

        print (term.clear())
        print ("User".ljust(30), "Events".ljust(6), "PRs".ljust(5), "Issues".ljust(6), "Pushes".ljust(7))
        sorted_users = sorted(users.items(), key = lambda kv: (kv[1], kv[0]), reverse=True)
        for i in range(20):
            u = sorted_users[i]
            ue = users_events[u[0]]
            print(u[0].ljust(30), str(u[1]).ljust(6), 
                  str(ue.get('PullRequestEvent', '')).ljust(5), 
                  str(ue.get('IssuesEvent', '')).ljust(6), 
                  str(ue.get('PushEvent', '')).ljust(7))
        time.sleep(1)

In [19]:
#export
def push_to_log(e):
    return f"{e.actor.login} pushed {len(e.payload.commits)} commits to repo {e.repo.name}"

In [20]:
#export
def str_clean(s): return s[:95]

In [21]:
#export
def signal_handler(sig, frame):
    if sig != signal.SIGINT: return
    print(term.exit_fullscreen())
    print(term.clear())
    print(term.normal)
    sys.exit(0)

In [22]:
#export
def quad_logs():
    term.enter_fullscreen()
    ui = HSplit(
            VSplit(
                Log(title='Issues', border_color = 2, color=7),
                Log(title='Commits', border_color = 2, color=3)
            ),
            VSplit(
                Log(title='Pull Requests', border_color = 2, color=4),
                Log(title='Releases', border_color = 2, color=5)
            ),
        )

    issues,commits = ui.items[0].items
    prs,releases = ui.items[1].items
    for o in issues,commits,prs,releases: o.append(" ")

    while True:
        for x in fetch_events():
            t = x["type"]
            if t == 'PushEvent': commits.append(str_clean(push_to_log(x)))
            elif t == 'IssuesEvent' or t == 'IssueCommentEvent': issues.append(str_clean(to_log(x)))
            elif t == 'PullRequestEvent': prs.append(str_clean(to_log(x)))
            elif t == 'ReleaseEvent': releases.append(str_clean(to_log(x)))
        ui.display()
        time.sleep(0.1)

In [23]:
#export
def simple():
    while True:
        for x in fetch_events(): print(f"{x.actor.login} {x.type} {x.repo.name}")

In [24]:
#export
help_msg = "Usage: ghtop <tail|quad|users|simple>"
mode_map = defaultdict(lambda: exit(help_msg),{'tail': tail_events,
                                              'quad': quad_logs,
                                              'users': watch_users,
                                              'simple': simple})

if __name__ == '__main__' and not in_jupyter():
    if len(sys.argv) < 2: exit(help_msg)
    _, mode, *_ = sys.argv
    signal.signal(signal.SIGINT, signal_handler)
    mode_map[mode]()